# The QNMs of Kerr black holes

This notebook compute the quasinormal modes of Kerr black holes: the QN frequencies, the angular and radial functions. We use Leaver's units: M=1/2 here.

In [ ]:
(*We first set the parameters of the Kerr spacetime, the field and \
the mode we're interested in:*) 
ar = 0.0 ; (*remember Stein's spin is 1/2 Leaver's spin!*)
br = Sqrt[1 - 4*ar^2]; 
rminus = (1 - br)/2; 
rplus = (1 + br)/2; 
s = -2; 
m = 0; 
l = 3; 
k1 = 1/2*Abs[m - s]; 
k2 = 1/2*Abs[m + s]; 
Alm = l*(l + 1) - s*(s + 1);

In [ ]:
winit = 0.9403 - 1.8312*I(*This is an initialization for the frequency*)

In [ ]:
(*In the following we compute the continued fraction with 130 terms \
(you can do it with more of course) and recursively re-computing the \
angular separation constant. Seven recursions are enough for this \
value of rotation parameter, as seen in the iterated values; See eqs. \
(18)-(27) in [1]*)

ie = 0; 
While[ie < 100, NITMAX = 150; 

c0 = 1 - s - I*w - 2*I/br*(w/2 - ar*m); 
c1 = -4 + 2*I*w*(2 + br) + 4*I/br*(w/2 - ar*m); 
c2 = s + 3 - 3*I*w - 2*I/br*(w/2 - ar*m); 
c3 = w^2*(4 + 2*br - ar^2) - 2*ar*m*w - s - 1 + (2 + br)*I*w - Alm + (4*w + 2*I)/br*(w/2 - ar*m); 
c4 = s + 1 - 2*w^2 - (2*s + 3)*I*w - (4*w + 2*I)/br*(w/2 - ar*m); 

\[Gamma] = Function[n, n^2 + (c2 - 3)*n + c4 - c2 + 2];
\[Beta] = Function[n, -2*n^2 + (c1 + 2)*n + c3];
\[Alpha] = Function[n, n^2 + (c0 + 1)*n + c0];

Leaver31[w_] := Module[{Rn}, For[{n = NITMAX; Rn = -1.0;}, n > 0,
    {
     Rn = \[Gamma][n]/(\[Beta][n] - \[Alpha][n]*Rn); n--;}]; Rn]; 
Leaver33[w_] := \[Beta][0]/\[Alpha][0] - Leaver31[w]; 

wang = FindRoot[Leaver33[w] == 0, {w, winit}][[1]][[2]]; 
witer[ie] = wang;

gammaang = Function[n, 2*ar*wang*(n + k1 + k2 + s)];
betaang = Function[n, n*(n - 1) + 2*n*(k1 + k2 + 1 - 2*ar*wang) - (2*ar* wang*(2*k1 + s + 1) - (k1 + k2)*(k1 + k2 + 1)) - (ar^2*wang^2 +s*(s + 1) + Sep)];
alphaang = Function[n, -2*(n + 1)*(n + 2*k1 + 1)]; 
 
Leaver31Ang[Sep_] := Module[{Rn},
   For[{n = NITMAX; Rn = -1.0;},
    n > 0,
    {
     Rn = gammaang[n]/(betaang[n] - alphaang[n]*Rn); n--;}
    ]; Rn]; 
    
Leaver33Ang[Sep_] := betaang[0]/alphaang[0] - Leaver31Ang[Sep];
Alm = FindRoot[Leaver33Ang[Sep] == 0, {Sep, Alm}][[1]][[2]]; 
ie = ie + 1];

In [ ]:
pr[n_] := NumberForm[n, Infinity, ExponentFunction -> (Null &)]
pr[wang*(10^10)/(10^10)] 
pr[Alm*(10^10)/(10^10)] 

In [ ]:
Table[witer[n], {n, 0, 10}]

In [ ]:
(* obtaining the coefficients of the expansions constructing ansatzes*)

Sep = Alm;
Nmax = 150; 
w = wang; 
sima = (w*rplus - ar*m)/br; 
ae[0] = 1; 
ae[1] = -betaang[0]/alphaang[0];
i = 1; 
While[i < Nmax, 
 ae[i + 1] = (-betaang[i]*ae[i] - gammaang[i]*ae[i - 1])/(alphaang[i]); 
i = i + 1];

In [ ]:
G0 = 4 ar^2 (u^2 - 1) \[Omega]^2 - 4 ar (u^2 - 1) \[Omega] ((u - 1) Abs[m - s] + (u + 1) Abs[m + s] + 2 (s + 1) u) + 4 (Alm (u^2 - 1) + m^2 + 2 m s u + s ((s + 1) u^2 - 1)) - 2 (u^2 - 1) Abs[m + s]- 2 (u^2 - 1) Abs[m - s] (Abs[m + s] + 1) - (u - 1)^2 Abs[m - s]^2 - (u + 1)^2 Abs[m + s]^2;
G1 = -8 ar (u^2 - 1)^2 \[Omega] - 4 (u^2 - 1) ((u - 1) Abs[m - s] + (u + 1) Abs[m + s] + 2 u);
G2 = -4 (u^2 - 1)^2;

In [ ]:
(* derivatives of g[u] w.r.t. u *)
g[u_, \[Omega]_, A_] := Sum[ae[i]*(1 + u)^(i), {i, 0, Nmax - 1}] /. u -> u; (* g[u] *)
gu[u_, \[Omega]_, A_] := Simplify[Dt[g[u, \[Omega], A], u, Constants -> {\[Omega]}]]; (* (g^\[Prime])[u] *)
guu[u_, \[Omega]_, A_] := Simplify[Dt[gu[u, \[Omega], A], u, Constants -> {\[Omega]}]]; (* (g^\[Prime]\[Prime])[u] *)

In [ ]:
N[G2*guu[u, \[Omega], A] + G1*gu[u, \[Omega], A] + G0*g[u, \[Omega], A] /. {u -> -1, \[Omega] -> wang, A -> Alm}]
GODE = G2*guu[u, \[Omega], A] + G1*gu[u, \[Omega], A] + G0*g[u, \[Omega], A] /. {\[Omega] -> wang, A -> Alm};

In [ ]:
function1 = Table[{u, Re[g[u, \[Omega], A] /. {\[Omega] -> wang, A -> Alm}], Im[g[u, \[Omega], A] /. {\[Omega] -> wang, A -> Alm}]}, {u, -0.999, 1, .001}]; 
function1a = Table[{u, Re[GODE /. {\[Omega] -> wang, A -> Alm}], Im[GODE /. {\[Omega] -> wang, A -> Alm}]}, {u, -0.999, 1, .001}];(* generate a table of 10000 points in u\[Element][-1, 1] and g[u]*)
Export["S_a0.4ell2.0m2.0n2.0.csv", function1, "CSV"]
Export["GODE_a0.4ell2.0m2.0n2.0.csv", function1a, "CSV"]

 Radial wavefunctions

In [ ]:
Nmax = 150; 
w = wang;
sima = (w*rplus - ar*m)/br; 
ae[0] = 1; 
ae[1] = -\[Beta][0]/\[Alpha][0]; 
i = 1; 
While[i < Nmax, 
ae[i + 1] = (-\[Beta][i]*ae[i] - \[Gamma][i]*ae[i - 1])/(\[Alpha][i]); 
i = i + 1];

In [ ]:
F0 = -ar^4 x^2 \[Omega]^2 - 2 ar^3 m x^2 \[Omega] + ar^2 (-A x^2 + x^2 (4 (rplus + 1) \[Omega]^2 + 2 I (rplus + 2) \[Omega] + 2 I s (\[Omega] + I) - 2) + x \[Omega]^2 - \[Omega]^2) + 2 ar m (rplus x^2 (2 \[Omega] + I) - x (\[Omega] + I) - \[Omega]) + A (x - 1) - I rplus (2 \[Omega] + I) (x^2 (s - 2 I \[Omega] + 1) - 2 (s + 1) x + 2 I \[Omega]) + (s + 1) (x - 2 I \[Omega]);
F1 = 2 ar^4 x^4 (x - I \[Omega]) - 2 I ar^3 m x^4 + ar^2 x^2 (2 rplus x^2 (-1 + 2 I \[Omega]) - (s + 3) x^2 + 2 x (s + I \[Omega] + 2) - 4 I \[Omega]) + 2 I ar m (x - 1) x^2 + (x - 1) (2 rplus x^2 (1 - 2 I \[Omega]) + (s + 1) x^2 - 2 (s + 1) x + 2 I \[Omega]);
F2 = ar^4 x^6 - 2 ar^2 (x - 1) x^4 + (x - 1)^2 x^2;

In [ ]:
(* derivatives of f[x] w.r.t. x *)
f[x_, \[Omega]_] := Sum[ae[i]*((rplus/x - rplus)/(rplus/x - rminus))^(i), {i, 0, Nmax - 1}]; (* f[x] *)
fx[x_, \[Omega]_] := Simplify[Dt[f[x, \[Omega]], x, Constants -> {\[Omega]}]]; (* (f^\[Prime])[x] *)
fxx[x_, \[Omega]_] := Simplify[Dt[fx[x, \[Omega]], x, Constants -> {\[Omega]}]]; (* (f^\[Prime]\[Prime])[x] *)

In [ ]:
N[F2*fxx[x, \[Omega]] + F1*fx[x, \[Omega]] + F0*f[x, \[Omega]] /. {x -> 0.5, \[Omega] -> wang, A -> Alm}]
RODE = F2*fxx[x, \[Omega]] + F1*fx[x, \[Omega]] + F0*f[x, \[Omega]] /. {\[Omega] -> wang, A -> Alm};

In [ ]:
function2 = Table[{x, Re[f[x, \[Omega]] /. {\[Omega] -> wang}], Im[f[x, \[Omega]] /. {\[Omega] -> wang}]}, {x, 0.1, 0.999, .001}]; 
function2a = Table[{x, Re[RODE /. {\[Omega] -> wang, A -> Alm}], Im[RODE /. {\[Omega] -> wang, A -> Alm}]}, {x, 0.1, 0.999, .0005}];(* generate a table of 10000 points in x \[Element] \[0, 0.95] and f[x]*)
Export["R_a0.4ell2.0m2.0n2.0.csv", function2, "CSV"]
Export["RODE_a0.4ell2.0m2.0n2.0.csv", function2a, "CSV"]

In [ ]:
(*A sanity check is to compare with the direct numerical integration of Teukolsky's wavefunction, with the same boundary conditions at the \
horizon:*)

In [ ]:
r1 = rplus + 10^(-6) 
r2 = 150

Delta = (r - rplus)*(r - rminus);
Kr = w*(r^2 + ar^2) - ar*m;
VgLe = 2*I*s*w*r - ar^2*w^2 - Alm + 1/Delta*((r^2 + ar^2)^2*w^2 - 2*ar*m*w*r + ar^2*m^2 + I*s*(ar*m*(2*r - 1) - w*(r^2 - ar^2)));
Vg = (Kr^2 - 2*I*s*Kr*(r - 1/2))/Delta + 4*I*s*w*r - Alm + 2*ar*w*m - ar^2*w^2;
sol = NDSolve[{Delta*D[y[r], {r, 2}] + (s + 1)*(2*r - 1)*D[y[r], {r, 1}] + Vg*y[r] == 0, 
   y[r1] == (rplus - rminus)^(-1 - s + I*w + I*sima)*Exp[I*w*r1]*(r1 - rplus)^(-s - I*sima), 
   Derivative[1][y][r1] == (rplus - rminus)^(-1 - s + I*w + I*sima)*Exp[I*w*r1]*(-s - I*sima)*(r1 - rplus)^(-s - I*sima - 1)}, 
   y, {r, r1, r2}, AccuracyGoal -> 20, PrecisionGoal -> 20, WorkingPrecision -> 30, MaxSteps -> 340000];
   
y[10] /. sol 